# Part 1: Modeling Text

### TF-IDF


First, we download the review.json file from the Resources tab on Piazza, a collection of about 7,000 Yelp reviews we sampled from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge). Each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field. We will load the json file first. We already have done some basic preprocessing on the reviews, so we can just tokenize each review using whitespace.

`TF = number of times word occurs in a review`

`IDF = log(total number of review / number of reviews containing the word)`

### A) Ranking with simple sums of TF-IDF scores

To start out with, for a multi-word query, we rank documents by a simple sum of the TF-IDF scores for the query terms in the review. 

In [22]:
import json
import math
from queue import PriorityQueue
from collections import Counter
from collections import deque

#  PreProcess Data - calculating IDF for all words in all reviews
idf = dict()  # Stores the idf of each word
reviews_with_word = dict()  # Stores the count of word in all the reviews. This is req for tf calculation
totalreviews = 0
total_words_inallreviews = 0
with open("review.json", "r") as json_file:
    totalreviews = sum(1 for line in json_file)

with open("review.json", "r") as json_file:
    for line in json_file:
        cur_review = json.loads(line)
        total_words_inallreviews += len(list(cur_review["review"].split()))
        unique_words = set(cur_review["review"].split())
        for word in unique_words:
            reviews_with_word[word] = reviews_with_word.get(word, 0) + 1

for word in reviews_with_word:
    idf[word] = math.log(totalreviews/reviews_with_word.get(word))

averagereviewlength = total_words_inallreviews/totalreviews


def calculate(listofwords, numberofreviews_to_display, scoring):

    # For each review in json file , calculate tf + idf score and store in queue
    q = PriorityQueue()
    with open("review.json", "r") as json_file:
        for line in json_file:
            cur_review = json.loads(line)

            # Calculate tf idf score for each word in list
            tfidf_score = 0  # Initialise tfidf score for current review
            docmagnitude = 0  # For cosine scoring , docmagnitude is sum of (tfidf of all words in current review) ** 2
            doc_vector = list()  # For cosine scoring , list of tfidf of all query words, in the current review
            query_vector = list()  # For cosine scoring , list of count of words in the query. e.g best best bbq -> [2 1]
            bm25score = 0
            length_of_current_review = 0
            wordsincur_review = list(cur_review["review"].split())
            if scoring == "bm25":
                length_of_current_review = len(wordsincur_review)
            if scoring == "cosine":
                for word in wordsincur_review:
                    tf = wordsincur_review.count(word)
                    tfidf = tf * idf[word]
                    docmagnitude += tfidf ** 2

            for word in listofwords:
                tf = wordsincur_review.count(word)
                tfidf = tf * idf[word]  # tfidf for current word in current review
                tfidf_score += tfidf    # tfidf score for all words in current review
                if scoring == "cosine":
                    query_vector.append(listofwords.count(word))
                    doc_vector.append(tfidf)
                if scoring == "bm25":
                    bm25score += getbm25(tf, idf[word], length_of_current_review)

            score = 0
            if scoring == "tfidf":
                score = tfidf_score
            elif scoring == "cosine":
                score = getcosine(doc_vector, query_vector, docmagnitude)
            elif scoring == "bm25":
                score = bm25score

            # Save top reviews in a queue
            if q.qsize() < numberofreviews_to_display:
                q.put((score, cur_review["review"]))

            else:
                minscore = q.get()
                if tfidf_score > minscore[0]:
                    q.put((score, cur_review["review"]))
                else:
                    q.put(minscore)
    return q


def getbm25(tf, idf, length_of_current_review):
    """
    k = 1.2
    b = 0.75
    bm25 = idf * tf * (1+k) / tf + (k * ( 1 - b + (b * (|D| / avgdl) ) ) )
    :param tf: tf score
    :param idf: idf score
    :param length_of_current_review: length
    :return:bm25
    """
    k = 1.2
    b = 0.75
    bm25 = (idf * (tf * (1+k))) / (tf + (k * (1 - b + (b * (length_of_current_review / averagereviewlength)))))
    return bm25


def getcosine(doc_vector, query_vector, docmagnitude):
    """
    Cosine(query, document) = DotProduct(query, document) / || query || * || doc ||
    :param doc_vector: For cosine scoring , list of tfidf of all query words, in the current review
    :param query_vector: For cosine scoring , list of count of words in the query. e.g best best bbq -> [2 1]
    :param docmagnitude: For cosine scoring , docmagnitude is sum of (tfidf of all words in current review) ** 2
    :return: Cosine value
    """
    if len(doc_vector) == 0 or len(query_vector) == 0:
        return 0

    dot_product = 0
    for i in range(len(doc_vector)):
        dot_product += doc_vector[i] * query_vector[i]

    mag_query = math.sqrt(sum(num**2 for num in query_vector))
    mag_doc = math.sqrt(docmagnitude)
    cos = dot_product / (mag_query * mag_doc)
    return cos


def score_documents(testquery: str, scoring: str, num_of_reviews_to_display: int):
    listofwords = testquery.split()
    top_reviews = calculate(listofwords, num_of_reviews_to_display, scoring)
    stack = deque()
    while not top_reviews.empty():
        stack.append(top_reviews.get())
    while stack:
        score, review = stack.pop()
        print(score)
        print(review)
        print("\n")


In [23]:
score_documents("best bbq", "tfidf", 2)

26.319774733191345
actually i d put it at 4 1 2 stars not perfect but prety good food can t say i m an alice cooper fan but we did see this on man vs food and the opportunity came up to go there we stopped in mid week for lunch not crowded the bar tender was great attentive personable good recommendations on food but not intrusive my wife had the lady gaga sausage appetizer fantastic sausage spicy great sauce served with celery stalks and bleu cheese optional she likes a good sausage let s leave it at that and alice cooper delivered since the bartender claimed the place is a bbq house i went with a bbq beef sandwich with chili and fries for the sides while the pig in me would have wanted more of everything it was the perfect size for lunch the brisket was done perfectly we travel with a smoker so i know a little about bbq the bbq sauce is very good i d like it spicier but i always do the sauce is multidimensional with some real depth to the bench it s not a one trick pony bbq sauce the

In [24]:
score_documents("kid fun and food", "tfidf", 2)

22.201231889774093
well it s probably all been said and been done about this place today i finally made the trip inside the store rather than just going to the restaurant on the side it was too early for lunch at tradiciones so in i went i knew that the market had a lot going on at all times i nearly run down at least 6 people getting in and out of the parking lot and i m a really good driver seriously not raymond from rain man good but actually good the parking lot on the store side is always packed so the focus of my trip was the cucina it was bustling for lunch already at just 10 30 so as i m trying to make sense of the menus posted in both spanish and english on the wall above the hot food line or whatever i am greeted by a nice woman that helps me decide where to go she mentioned sopes first so that s what i started with my order was way too much food for one but i wanted to try a couple of things so i got both a chicken and a steak sopa a street taco al pastor pork and a side of 

### B) Ranking with TF-IDF + Cosine

Instead of using the sum of TF-IDF scores, let's try the classic cosine approach for ranking. You should still use the TF-IDF scores to weigh each term, but now use the cosine between the query vector and the document vector to assign a similarity score. You can try the same two queries as before and report the results. (Top-10 reviews which are similar to the query)

In [25]:
score_documents("best bbq", "cosine", 2)

0.42640446015289
best food best service great prices 


0.02062166353352264
one of my all time favorite spots the basement bar is fantastic the atmosphere the service the music the fireplace the comfortable bar stools the clientele are all great generally my wife and i have gone here for reverse hh 5 wine and 5 small plates you cannot miss with the tri color roasted pepper bruschetta truth be told we rarely stick to just the hh items i can t resist the rokerij salad a magical wedge get it with the turkey and a good g t a nice pour and a 4 5 oz bottle of fever tree tonic if hh isn t your thing go for brunch or dinner as we finally did last sunday a group of six of us came in for dinner aside from an awesome meal and great service thank you jamie we lucked into coming in on a day when they were serving green chile stew spice tastic and 1 2 off bottles of wine what it was fantastic my wife loved the smoked turkey and the green chile pork enchiladas thanks richardson s and another member o

In [26]:
score_documents("kid fun and food", "cosine", 2)

0.38678092826505023
fun and friendly staff 


0.0010161170323554375
we could not have been happier i took my broken globe from an antique lamp to wizard of odz not really believing i would ever find a globe to match the second one on the lamp the woman who greeted us was great looked at the globe took off across the store wanted to know how many then brought one had three that was an exact match the cost was 10 could have chaged so much more we saw some very negative reviews and wanted to make certain that we share our very positive experience nyla dave




### C) Ranking with BM25

Refered [https://en.wikipedia.org/wiki/Okapi_BM25](https://en.wikipedia.org/wiki/Okapi_BM25) for the specific formula. Chose k_1 = 1.2 and b = 0.75.


In [27]:
score_documents("best bbq", "bm25", 2)


9.721870551819865
hmmm just having read all the reviews so far i m wondering if i liked this place as much as i think i did i do recall the bbq sauce being a little but too spicy for my taste but i think overall i really liked my bbq sandwich i ate at honey bear s or wait did i have the ribs shoot i think i had both it wasn t my favorite but it was good and i d go back again but i don t know bbq like some of you reviewers know bbq so maybe i m not the best review to take into consideration 


4.991527303398323
i was in town for training at epic wanted a bar with good food to watch baseball playoffs i remembered the nitty gritty near campus from a previous visit so tried the one in middleton close to my hotel bbq pulled pork sandwich outstanding waffle fries excellent capital amber draft very good atmosphere fun service great too bad it wasn t my birthday it seemed that everyone there had a birthday celebration in the group that s their thing i ll go back even if it s not my birthday 



In [28]:
score_documents("kid fun and food", "bm25", 2)

7.921666022538528
a friend surprised me with a trip here one day i decided to try out their silver admission cuz they had a check in offer omg so much fun for less than half the price of going to a big 6 flags or something you can get most of the same stuff here with rarely any lines when i went in the evening there were no lines honestly with the amount of time wasted at big parks in lines you get sooo much more here water rides coasters arcade huge mini golf course i only got half way and a zip line i felt like i was a kid again 


1.5458654633857807
was not feeling well yesterday so i ordered some food online for delivery from chang jiang originally i though i had enough cash to pay for the order but realized as i was finishing my online order i d have to pay by credit card i took the 23 and threw on the table next to the door so i could tip in cash when the guy delivered btw he was in my neighbors driveway about 2 feet into his lawn i grabbed what i though was a 3 tip woke up this 

A discussion on the three methods:

TF*IDF method:
It is the simplest method to implement. However the top match for "best bbq" was just a review with "bbq" ie the rarer term, mentioned 6 times which is to be expected given that rarer terms have large IDF. There was no penalty for a longer document, top scoring doc is a 358 word review, it scores high only because of the rarer term mentioned more number of times, however this approch will skew results towards longer documents which will obviously mention a term more often. An approach with some doc normalisation will give better results.

Cosine Method: 
This method takes care of document normalisation by using the doc_mag (the length of the doc to normalise the tf*idf score).
Takes more computation because tf idf score is to be calculated for each term in each doc for normalisation.
Cosine method's highest rated review for "best bbq" is just a short document with one term ie "best" even though "best" has lower idf than "bbq". Short doc length is over weighted/preferred in cosine approach. This approach severely penalises documents that are slightly longer and chooses one of the shortest docs as the best.

BM-25: 
Seems to be the best method in terms of computation cycles and extracting most important document.
Does not involve either the computation intensive ways of cosine doc normalisation (normalisation is with simple doc length) nor is it heavily skewed in favor of rarer terms.
Best rated document for "best bbq" was a relatively short doc length of 110 words, with bbq freq=4 and best freq=1. This was the most pertinent and relevant doc for the given query.